In [148]:
import matplotlib.pyplot as plt
import numpy as np 
import math

In [176]:
# Functions to calculate alpha and beta for each ion at a given voltage
def alpha_m(V):
  if (V == 25):
    return alpha_m(V+.01)
  return 0.1*(25-V)/(math.exp((25 - V)/10)-1)
def beta_m(V): 
  return 4*math.exp(-1*V/18)
def alpha_h(V):
  return 0.07*math.exp(-1*V/20)
def beta_h(V):
  if (V == 3):
    return beta_h(V+0.01)
  return 1/(math.exp((30-V)/10)+1)
def alpha_n(V):
  if (V == 10):
    return alpha_n(V+0.01)
  return (0.01*(10-V))/(math.exp((10-V)/10)-1)
def beta_n(V):
  return 0.125*math.exp(-1*V/80)

# Define constants 
C = 1
gk_max = 36; gna_max = 120; gl_max = 0.3;
Ek = -12; Ena = 115; El = 10.6;
Ri = 0.035

# Functions that return the value of a derivative 
def V_prime(i, current):
  I = 0
  if (i < 1000):
    I = current
  return (1/C)*(gna_max*m[i-1]**3*h[i-1]*(Ena-V[i-1])+gk_max*n[i-1]**4*(Ek-V[i-1])+gl_max*(El-V[i-1])+I)
def m_prime(i):
  return alpha_m(V[i-1])*(1-m[i-1])-beta_m(V[i-1])*m[i-1]
def n_prime(i): 
  return alpha_n(V[i-1])*(1-n[i-1])-beta_n(V[i-1])*n[i-1]
def h_prime(i):
  return alpha_h(V[i-1])*(1-h[i-1])-beta_h(V[i-1])*h[i-1]

u_d = 0.05; u_dx = 0.1
u_const = u_d/(4*Ri**u_dx**2)
# Unmyelinated derivative functions
def u_V_prime(t, k, current):
  I = 0
  if (t < 1000):
    I = current
  if (k == 0):
    return (1/C)*(u_const*(V[t][k+1]-V[t][k]) + gna_max*m[t][k]**3*h[t][k]*(Ena-V[t][k])+gk_max*n[t][k]**4*(Ek-V[t][k])+gl_max*(El-V[t][k])+I)
  if (k == 99):
    return (1/C)*(u_const*(V[t][k-1]-V[t][k]) + gna_max*m[t][k]**3*h[t][k]*(Ena-V[t][k])+gk_max*n[t][k]**4*(Ek-V[t][k])+gl_max*(El-V[t][k])+I)
  return (1/C)*(u_const*(V[t][k+1]+V[t][k-1]-2*V[t][k]) + gna_max*m[t][k]**3*h[t][k]*(Ena-V[t][k])+gk_max*n[t][k]**4*(Ek-V[t][k])+gl_max*(El-V[t][k])+I)
def u_m_prime(t, k):
  return alpha_m(V[t][k])*(1-m[t][k])-beta_m(V[t][k])*m[t][k]
def u_n_prime(t, k): 
  return alpha_n(V[t][k])*(1-n[t][k])-beta_n(V[t][k])*n[t][k]
def u_h_prime(t, k):
  return alpha_h(V[t][k])*(1-h[t][k])-beta_h(V[t][k])*h[t][k]

In [173]:
# Create 2D arrays that will hold V(t,k), m(t,k), h(t,k), n(t,k)
segments = 100 
time = np.arange(0, 25, .001)
rows, cols = len(time), segments 
V = [[0 for t in range(cols)] for k in range(rows)]
m = [[0 for t in range(cols)] for k in range(rows)]
h = [[0 for t in range(cols)] for k in range(rows)]
n = [[0 for t in range(cols)] for k in range(rows)]
print(len(V[0]))

100


In [177]:
# Initialize each segment with starting values 
for k in range(0, segments): 
  V[0][k] = 0
  m[0][k] = 0.053
  n[0][k] = 0.318
  h[0][k] = 0.596

dt = 0.001 
first_current = 200
# Iterate through time and space
for t in range(1, 2):
  for k in range(1, 4):
    V[t][k] = V[t-1][k] + dt * u_V_prime(t-1, k, first_current)
    m[t][k] = m[t-1][k] + dt * u_m_prime(t-1, k)
    n[t][k] = n[t-1][k] + dt * u_n_prime(t-1, k)
    h[t][k] = h[t-1][k] + dt * u_h_prime(t-1, k)
#17 seconds first try 
print(V[0])
print(V[1])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0.199986824171168, 0.199986824171168, 0.199986824171168, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [144]:
print(V[0])
print(V[1])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0.199986824171168, 0.199986824171168, 0.199986824171168, 0.199986824171168, 0.199986824171168, 0.199986824171168, 0.199986824171168, 0.199986824171168, 0.199986824171168, 0.199986824171168, 0.199986824171168, 0.199986824171168, 0.199986824171168, 0.199986824171168, 0.199986824171168, 0.199986824171168, 0.199986824171168, 0.199986824171168, 0.199986824171168, 0.199986824171168, 0.199986824171168, 0.199986824171168, 0.199986824171168, 0.199986824171168, 0.199986824171168, 0.199986824171168, 0.199986824171168, 0.199986824171168, 0.199986824171168, 0.199986824171168, 0.199986824171168, 0.199986824171168, 0.199986824171168, 0.199986824171168, 0.199986824171168, 0.199986824171168, 0.199986824

In [139]:

rows, cols = 5, 10 
V = [[0 for t in range(cols)] for k in range(rows)]
V[2][1] = 7
V
temp_row = V[2]
temp_row

[0, 7, 0, 0, 0, 0, 0, 0, 0, 0]

In [77]:
print(V[500])

[0, 1.2904272982864116, 1.2904272982864116, 1.2904272982864116, 1.2904272982864116, 1.2904272982864116, 1.2904272982864116, 1.2904272982864116, 1.2904272982864116, 1.2904272982864116, 1.2904272982864116, 1.2904272982864116, 1.2904272982864116, 1.2904272982864116, 1.2904272982864116, 1.2904272982864116, 1.2904272982864116, 1.2904272982864116, 1.2904272982864116, 1.2904272982864116, 1.2904272982864116, 1.2904272982864116, 1.2904272982864116, 1.2904272982864116, 1.2904272982864116, 1.2904272982864116, 1.2904272982864116, 1.2904272982864116, 1.2904272982864116, 1.2904272982864116, 1.2904272982864116, 1.2904272982864116, 1.2904272982864116, 1.2904272982864116, 1.2904272982864116, 1.2904272982864116, 1.2904272982864116, 1.2904272982864116, 1.2904272982864116, 1.2904272982864116, 1.2904272982864116, 1.2904272982864116, 1.2904272982864116, 1.2904272982864116, 1.2904272982864116, 1.2904272982864116, 1.2904272982864116, 1.2904272982864116, 1.2904272982864116, 1.2904272982864116, 1.29042729828641